<div class="alert alert-info" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<font size="5"><b><center>Yandex Hackathon: Preprocessing by Team № 4</center></b></font>

**Данный ноутбук посвящён предообработке данных, в нём будет указана методика по которой из сырых данных можно получить данные для обучения моделей. При обработке данных необходимо учесть несколько пунктов:**

 - **Выбрана двухуровневая модель (кластеризация + классификация). При этом кластеризация должна будет разбить товары на лкастеры, а поэтому должна учиться на всех доступных товарах, в то время как классификация может учиться только на размеченом датасете**
  - **При исследовании данных, было установлено, что в данных есть противоречия по целевому признаку: для одного и того же товара могут быть использованны разные типы коробок. Данный момент необходимо будет обработать, поскольку иначе модель не научится корректно выбирать правильные ответы**
 - **Карготипы товаров указаны в виде кодов и их необходимо преобразовать в удобный для ML моделей вид**

**Начнём с импора требуемых библиотек и указаний путей**

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm, notebook

import os.path
import os
import pickle
import joblib
import sys

import scipy
from scipy.sparse import csr_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import RobustScaler

import warnings

warnings.filterwarnings("ignore")
notebook.tqdm().pandas()

0it [00:00, ?it/s]

**Укажем пути к "сырым" данным**

In [2]:
cwd = os.getcwd()

In [3]:
cwd

'/Users/greygreywolf/Documents/yandex_market_hackaton_team_4'

In [4]:
PATH_DATA = cwd + '/initial_data/data.csv'
PATH_SKU = cwd + '/initial_data/sku.csv'
PATH_SKU_CARGOTYPES = cwd + '/initial_data/sku_cargotypes.csv'
PATH_CARTON = cwd + '/initial_data/carton.csv'
PATH_CARGOTYPES_INFO = cwd + '/initial_data/cargotype_info.csv'
PATH_CARTON_PRICE = cwd + '/initial_data/carton_price.xlsx'

**Укажем пути к обработанным данным**

In [5]:
PATH_TO_SAVE_DF_FOR_CLASSIFIC = cwd + '/preprocessed_data/df_for_classific.csv'
PATH_TO_SAVE_REC_PRED = cwd + '/preprocessed_data/prediction_rec_sys.csv'
PATH_TO_SAVE_SPARSE_FOR_CLUSTER = cwd + '/preprocessed_data/sparse_cluster.npz'
PATH_TO_SAVE_SPARSE_FOR_CLUSTER_TO_CLASSIF = cwd + '/preprocessed_data/sparse_cluster_to_classif.npz'

**Укажем пути для сохранения нормализаторов и векторайзера**

In [6]:
PATH_TO_SAVE_SCALER_CLASIFIC = cwd + '/for_docker_images/src/scalers/scaler_rb_for_clusific.bin'
PATH_TO_SAVE_SCALER_FOR_CLUSTER = cwd + '/for_docker_images/src/scalers/scaler_rb_for_cluster.bin'
PATH_TO_SAVE_TFIDF_VECTORIZER = cwd + '/for_docker_images/src/vectorizers/tfidf_vectorizer.bin'

## Основной датасет

**Откроем основной датасет, датасет с sku, датасет с карготипами sku и размерами коробок**

In [7]:
df_data = pd.read_csv(PATH_DATA)
df_sku = pd.read_csv(PATH_SKU, index_col=0)
df_sku_cargotypes = pd.read_csv(PATH_SKU_CARGOTYPES)
df_carton = pd.read_csv(PATH_CARTON)

**Выведем размеры полученных датасетов**

In [8]:
print(df_data.shape) 
print(df_sku.shape)
print(df_sku_cargotypes.shape)
print(df_carton.shape)

(325623, 15)
(6385961, 4)
(21053152, 3)
(29, 5)


**Упорядочим габаритные размеры от меньшего к большему и почистим выбросы.**

In [9]:
def cleaning_sku(df):
    df['list_demensions'] = df.progress_apply(lambda x: sorted([x['a'], x['b'], x['c']]), axis=1)
    df['a'] = df.progress_apply(lambda x: x['list_demensions'][0], axis=1)
    df['b'] = df.progress_apply(lambda x: x['list_demensions'][1], axis=1)
    df['c'] = df.progress_apply(lambda x: x['list_demensions'][2], axis=1)
    df = df[df['a']<175]
    df = df[df['b']<190]
    df = df[df['c']<500]
    return df

In [10]:
df_sku = cleaning_sku(df_sku)

  0%|          | 0/6385961 [00:00<?, ?it/s]

  0%|          | 0/6385961 [00:00<?, ?it/s]

  0%|          | 0/6385961 [00:00<?, ?it/s]

  0%|          | 0/6385961 [00:00<?, ?it/s]

**Размер df_sku после чистки**

In [11]:
print(df_sku.shape)

(6384291, 5)


**Рассчитаем объём для каждого товара**

In [12]:
df_sku['volume'] = df_sku['a'] * df_sku['b'] * df_sku['c']

**Добавим столбец указываюший на отсутствие размеров**

In [13]:
df_sku['no_size'] = ((df_sku['a'].values==0) & 
                     (df_sku['b'].values==0) & 
                     (df_sku['c'].values==0)).astype('int')

**В первоначальном датасете оставим только те, которые есть в изменённом sku**

In [14]:
df_data = df_data[df_data['sku'].isin(df_sku['sku'])]

**Почистим датасет от товаров с нулевым весом**

In [15]:
df_data = df_data[df_data['goods_wght']!=0]

**Выведем получившийся размер датасета**

In [16]:
print(df_data.shape) 

(314755, 15)


**При обработке заказа, нам потребуется обучить модель группировать товары вместе и при этом корректно прогнозировать коробку. Однако для правильного прогноза требуется больше данных, чем есть в нашем распоряжении (нужно точно знать сколько штук каждого товара и в какие упаковки было упаковано в каждом заказе). Поэтому было принято решение обучить модель подбирать конкретную коробку под конкретный айтем, при этом при обучении не даватьь модели sku, а только указывать весо-габаритные характеристики и метрицу карготипов. В таком случае модель научится распределять товары по их характеристикам, а не по sku. А товары будут группироваться на основе класстеризации при помощи отдельного алгоритма, их характеристики будут объединяться и подаваться на вход модели как отдельный товар. Поэтому необходимо преобразовать датасет, чтобы он корректно предиктил требуемые упаковки.**

**Сделаем выборку по датасету, чтобы одному sku соответствовало 1 значение упаковки. Для этого:** 

 - **в функции cleaning_data:**   
     - **посчитаем количество коробок конкретного типа для каждого sku, которые вбирал упаковщих (count_sku_cart)**
     - **посчитаем общество количество коробок всех типа для каждого sku, которые вбирал упаковщих (count_sku_all)**
     - **рассчитаем для каждого сочетания конкретного типа коробки с sku, вероятность того, что упаковщих выбирает именно её (perc_carton = count_sku_cart / count_sku_all)**
     - **посчитаем объём коробок перемножив их габариты из df_cart**
     - **вызовем функцию get_one_carton**  
     
 - **в функции get_one_carton:**               
     - **Рассчитаем уникальные вхождения для каждого типа товара**
     - **Для товаров у которых есть разные вхождения выбираем те коробки у которых вероятность вхождения максимальна (например если было 10 коробок и при их упаковки были выбраны 2 упаковки одного типа, 1 упаковка другого типа и 6 упаковок третьего, т.е. вероятности равны 0,2, 0,1 и 0,6 соответственно, то для рекомендаций будет выбран третий тип упаковок)**
     - **При этом необходимо отсортировать те коробки у которых дублируется максимальная частота вхождений их потребуется обработать отдельно**
     - **Для товаров у которых одинаковая частота вхождения коробок будем выбирать ту, у которой  наименьший объём. Данное решение было принято в связи с тем, что для одного и того же товара объём коробок может отличаться в 4 раза и более (вероятнее всего в большую коробку складывали несколько экземляров товара, но мы не знаем сколько штук), поэтому с целью оптимизации будем брать наименьшую упаковку.**

In [17]:
def get_one_carton(df_cart, name_column):
    #рассчёт уникальных вхождений вероятностей для каждого товара
    df_perc_car = df_cart.groupby(['sku', 'perc_carton'])['count_sku_cart'].agg('count').reset_index(drop=False)
    dict_count_perc = {sku: df_perc_car[df_perc_car['sku']==sku]['sku'].count() for sku in df_perc_car['sku'].unique()}    
    df_cart['count_perc'] = df_cart['sku'].map(dict_count_perc)
    
    # Выбор для товаров у которых есть разные вхождения
    df = df_cart[df_cart['count_perc']!=1]
    df = df.groupby('sku')['perc_carton'].agg('max').reset_index(drop=False)
    df_final = df.merge(df_cart[['sku', 'perc_carton', name_column, 'volume']], how='left', on=['sku', 'perc_carton'])
    # Обработка тех у кого дублируются максимальные вхождения
    dupl_sku = df_final[df_final['sku'].duplicated()]['sku']
    df_dupl_max = df_final[df_final['sku'].isin(dupl_sku)]
    df_final = df_final[~df_final['sku'].isin(dupl_sku)]
    
    # Выбор для товаров с одинаковой частотой вхождения
    df = df_cart[df_cart['count_perc']==1]
    # Добавим дублирующиеся максимальные вхождения
    df = pd.concat((df, df_dupl_max), axis=0)
    df = df.groupby('sku')['volume'].agg('min').reset_index(drop=False)
    df = df.merge(df_cart[['sku', 'perc_carton', 'volume', name_column]], how='left', on=['sku', 'volume'])
    df_final = pd.concat((df_final, df), axis=0)
    return df_final[['sku', name_column]]

In [18]:
def cleaning_data(df_data_in, df_carton_in, name_column):
    df_cart = df_carton_in.copy(deep=True)
    selected_carton_sku = df_data_in.groupby(['sku', name_column])['sku'].agg('count')
    df_sel_cart_sku = (pd.DataFrame(selected_carton_sku).
                              rename(columns={'sku':'count_sku_cart'}).
                              reset_index(drop=False))
    df_count_sku = (pd.DataFrame(df_data_in.groupby('sku')['sku'].agg('count')).
                    rename(columns={'sku': 'count_sku_all'}).
                    reset_index(drop=False))
    df_sel_cart_sku = df_sel_cart_sku.merge(df_count_sku, how='left', on='sku')

    df_sel_cart_sku['perc_carton'] = df_sel_cart_sku['count_sku_cart'] / df_sel_cart_sku['count_sku_all']

    df_cart['volume'] = df_cart['LENGTH'] * df_cart['WIDTH'] * df_cart['HEIGHT']
    df_cart = df_cart.rename(columns={'CARTONTYPE': name_column})

    df_sel_cart_sku = df_sel_cart_sku.merge(df_cart[[name_column, 'volume']], how='left', on=name_column)
    
    return get_one_carton(df_sel_cart_sku, name_column)

In [19]:
%%time
df_cleaned = cleaning_data(df_data, df_carton, 'selected_carton')

CPU times: user 17min 48s, sys: 6.41 s, total: 17min 54s
Wall time: 18min 9s


**Выведем размерность итогового датасета**

In [20]:
df_cleaned.shape

(81149, 2)

**Посмотрим количество получившихся упаковок**

In [21]:
df_cleaned['selected_carton'].value_counts()

MYB        20733
MYC        14986
MYA         8053
MYD         6970
NONPACK     6591
YMC         4525
STRETCH     3775
YMA         3126
YMG         2893
YMF         2735
YMW         2500
MYE         2286
YME         1273
MYF          348
YML          251
YMX          103
YMB            1
Name: selected_carton, dtype: int64

**Удалим тип упаковки, который встреччается только 1 раза, модель не научится его корректно предсказывать**

In [22]:
df_cleaned = df_cleaned[df_cleaned['selected_carton']!='YMB']

**Проведём аналогичный рассчёт для столбца с рекомендованной упаковкой. Он нам потребуется для определения качества действующей модели.**

In [23]:
%%time
df_rec = cleaning_data(df_data, df_carton, 'recommended_cartontype')

CPU times: user 16min 53s, sys: 3.43 s, total: 16min 56s
Wall time: 17min 4s


In [24]:
df_rec = df_rec.merge(df_cleaned, how='inner', on='sku')

## Матрица карготипов (TfIdf)

**В данных приведены сведения о каргоипах товара, однако их необходимо преобразовать в вид, который сможет воспринимать Ml модель. Самым протым и очевидным вариантом было бы создать матрицу каргоипов по принципу OneHotEncoding, но такой подход не учитывал бы частоту встречания того или иного признака, поэтому было принято решения использова TfIdf**

**Создадим матрицу tfidf и присоединим к нашему датафрейму**

In [25]:
def forming_df_for_tfidf(df_in):
    #функция преобразует столбец в cargotype в формат string, и для каждого sku формирует строку из всех его карготипов
    df = df_in.copy(deep=True)
    df['cargotype'] = df['cargotype'].astype('str')
    df_for_tfidf = df[['sku', 'cargotype']]
    df_for_tfidf['cargotype'] = df_for_tfidf['cargotype'] + ' '
    df_for_tfidf = df_for_tfidf.groupby('sku')['cargotype'].agg('sum').reset_index(drop=False)
    return df_for_tfidf

In [26]:
df_tfidf = forming_df_for_tfidf(df_sku_cargotypes)

In [27]:
def get_tfidf_dataframe(df, path_to_save_vect):
    #функция производит векторизацию признаков при помощи tfidf и сохраняет обученный tfidf
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['cargotype'])
    dict_for_columns = {i[1]:i[0] for i in tfidf.vocabulary_.items()}
    new_df = pd.DataFrame.sparse.from_spmatrix(tfidf_matrix).rename(columns=dict_for_columns)
    new_df.insert(0, 'sku', df['sku'])
    pickle.dump(tfidf, open(path_to_save_vect, "wb"))
    return new_df

In [28]:
df_tfidf = get_tfidf_dataframe(df_tfidf, PATH_TO_SAVE_TFIDF_VECTORIZER)

**Присоединим матрицу с карготипами к датасету с габаритами sku.**

In [29]:
df_sku = df_sku[['sku', 'a', 'b', 'c', 'volume', 'no_size']].merge(df_tfidf, how='left', on='sku')

**Поскольку для некоторых sku не указаны карготипы, то заполним такие пропуски их нулями, предварительно создав столбец с меткой пропуска данной информации.**

In [30]:
df_sku['no_cargotype'] = (~df_sku['sku'].isin(df_tfidf['sku'])).astype('int')

In [31]:
df_sku = df_sku.fillna(0) 

## Датасет для классификации

**Подготвим датасет для классификации. присоединим к ранее выбранным sku полученный датасет с  габаритными размерами и карготипам**

In [32]:
df_for_classific = df_cleaned.merge(df_sku, how='left', on='sku')

**Добавим вес товара из основного датасета для этого создадим датасет sku - goods_wght, При этом необходимо учесть, что для некоорых товаров вес указывали разный (порядка 100 штук таких товаров). Такие товары также необходимо обработать.**

In [33]:
df_sku_wght = df_data.groupby(['sku', 'goods_wght'])['goods_wght'].agg(['count']).reset_index(drop=False)

**Создадим датасет, который будет показывать количество вариантов веса для каждого товара**

In [34]:
df_count_wght = df_sku_wght.groupby('sku')['goods_wght'].agg(['count']).reset_index(drop=False)

**Объединим товары у которых встречается разный вес в отдельный датасет**

In [35]:
df_sku_wght_bad = df_sku_wght[df_sku_wght['sku'].isin(df_count_wght[df_count_wght['count']>1]['sku'])]

In [36]:
df_sku_wght_bad

,sku,goods_wght,count
467,017a9e8665acbb0272675d8bc2e724a4,1.870,2
468,017a9e8665acbb0272675d8bc2e724a4,1.873,1
1086,036724225d7ff6f662afcbf909423c30,1.220,7
1087,036724225d7ff6f662afcbf909423c30,1.260,21
1695,053c9e45bf35a7f64dabcc1bcf46ff7f,1.850,25
...,...,...,...
79480,fc1b8ac0627826d2b50c3e08997f11c6,0.460,1
79572,fc74a8a114124ad54c2c75d887c4c4de,0.185,1
79573,fc74a8a114124ad54c2c75d887c4c4de,0.190,11
80656,ffd71f66b40c1bea7edcb427ffcfbef5,0.840,1


**Поскольку разница незначительная, то сгруппируем такие товары и возьмём среднее значение**

In [37]:
df_sku_wght_bad = df_sku_wght_bad.groupby('sku')['goods_wght'].agg('mean').reset_index(drop=False).rename(columns={'mean': 'goods_wght'})

**Выберем остальной датасет и удалим колонку с количеством вхождений**

In [38]:
df_sku_wght = df_sku_wght[df_sku_wght['sku'].isin(df_count_wght[df_count_wght['count']==1]['sku'])].drop('count', axis=1)

**Объединим оба датасета в один**

In [39]:
df_sku_wght = pd.concat((df_sku_wght, df_sku_wght_bad), axis=0)

**Присоединим полученные данные к датасет для классификации**

In [40]:
df_for_classific = df_for_classific.merge(df_sku_wght, how='left', on='sku')

**Добавим плотность**

In [41]:
df_for_classific['specific_weight'] = df_for_classific['goods_wght'] / df_for_classific['volume']
df_for_classific['specific_weight'] = df_for_classific['specific_weight'].fillna(0)

**Заменим значение бесконечности максимальным значением int**

In [42]:
df_for_classific.loc[df_for_classific['specific_weight']==np.inf, 'specific_weight'] = sys.maxsize

**Смаштабируем размеры. Для масштабирования будем использовать RobustScaler, поскольку он лучше борется с выбросами**

In [43]:
scaler_for_classif = RobustScaler()

In [44]:
df_for_classific[['a', 'b', 'c', 'volume', 'goods_wght', 'specific_weight']] = scaler_for_classif.fit_transform(
                                                                        df_for_classific[['a', 'b', 'c', 'volume', 'goods_wght', 'specific_weight']])

**Сохраним скейлер для классификации**

In [45]:
joblib.dump(scaler_for_classif, PATH_TO_SAVE_SCALER_CLASIFIC, compress=True)

['/Users/greygreywolf/Documents/yandex_market_hackaton_team_4/for_docker_images/src/scalers/scaler_rb_for_clusific.bin']

**Удалим столбец sku, переименуем столбец selected_carton в target**

In [46]:
df_for_classific = df_for_classific.drop('sku', axis=1)
df_for_classific = df_for_classific.rename(columns={'selected_carton': 'target'})

In [47]:
df_rec = df_rec.drop('sku', axis=1)

**Запишим полученные датасеты в файлы**

In [48]:
df_for_classific.to_csv(PATH_TO_SAVE_DF_FOR_CLASSIFIC, index=False)

In [49]:
df_rec.to_csv(PATH_TO_SAVE_REC_PRED, index=False)

In [50]:
df_for_classific

,target,a,b,c,volume,no_size,10,1010,1011,110,...,950,955,960,970,980,985,990,no_cargotype,goods_wght,specific_weight
0,YMG,0.800000,0.627273,0.414286,1.094040,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0,4.432258,1.280660
1,MYB,0.036364,-0.572727,-0.164286,-0.198518,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0,-0.138710,0.524116
2,MYB,-0.218182,-0.300000,-0.157143,-0.176471,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0,-0.325806,-0.241356
3,YMF,0.545455,-0.345455,-0.142857,0.008662,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0,-0.091935,-0.118520
4,YMA,0.363636,-0.436364,-0.714286,-0.203722,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.320382,0.0,0.0,0.0,0.0,0,0.080645,1.408694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81143,MYA,0.000000,-0.163636,-0.714286,-0.200513,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0,-0.322581,-0.151706
81144,MYC,-0.363636,-0.800000,0.357143,-0.288739,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0,-0.322581,0.589966
81145,MYA,-0.545455,-0.618182,-0.642857,-0.325634,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0,-0.322581,1.999755
81146,MYB,-0.181818,0.563636,-0.142857,0.054860,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0,0.000000,-0.056187


## Датасет для кластеризации

**Поскольку было принято решение использовать кластеризацию товаров, то потребуется создать отдельный датасет по всем товарам, а не только по тем, которые указаны в основном датасете.**

**Покольку, в отличии от датасета для классификации, в датасет для кластеризации могут попасть товары без линейных размеров, то добавим соответствующий столбец**

In [51]:
matrix_for_cluster = df_sku.copy(deep=True)

**Удалим столбец с sku**

In [52]:
matrix_for_cluster.drop(['sku'], axis=1, inplace=True)

**Аналогично с датасетом для классификации будем использовать RobustScaler, но при этом его нужно будет сохранить отдельным файлом, так как он обучался на других столбцах**

In [53]:
scaler_for_cluster = RobustScaler()

In [54]:
matrix_for_cluster[['a', 'b', 'c', 'volume']] = scaler_for_cluster.fit_transform(matrix_for_cluster[['a', 'b', 'c', 'volume']])

In [55]:
joblib.dump(scaler_for_cluster, PATH_TO_SAVE_SCALER_FOR_CLUSTER, compress=True)

['/Users/greygreywolf/Documents/yandex_market_hackaton_team_4/for_docker_images/src/scalers/scaler_rb_for_cluster.bin']

**Поскольку у нас очень много товаров, при этом матрица карготипов является разряженой, то разумнее сохранить её именно как разряженную матрицу, для этого напишем соответствующую функцию**

In [56]:
def save_sparse_csr(filename: str, array):
    np.savez(filename, data=array.data, indices=array.indices,
             indptr=array.indptr, shape=array.shape)

In [57]:
save_sparse_csr(PATH_TO_SAVE_SPARSE_FOR_CLUSTER, csr_matrix(matrix_for_cluster))

**Поскольку для лкассификации планируется использовать предсказанные кластера, то сделаем отдельный датасет для значений из кластеризации, поскольку scalerы у них должны быть разные**

In [58]:
matrix_for_cluster_to_classif = df_cleaned.merge(df_sku, how='left', on='sku')

In [59]:
matrix_for_cluster_to_classif[['a', 'b', 'c', 'volume']] = (scaler_for_cluster.
                                                            transform(matrix_for_cluster_to_classif[['a', 'b', 'c', 'volume']]))

In [60]:
matrix_for_cluster_to_classif.drop(['sku', 'selected_carton'], axis=1, inplace=True)

In [61]:
matrix_for_cluster_to_classif

,a,b,c,volume,no_size,10,1010,1011,110,120,...,930,931,950,955,960,970,980,985,990,no_cargotype
0,0.550,0.361538,0.23750,0.700005,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0
1,0.025,-0.653846,-0.26875,-0.177178,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0
2,-0.150,-0.423077,-0.26250,-0.162216,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0
3,0.375,-0.461538,-0.25000,-0.036577,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0
4,0.250,-0.538462,-0.75000,-0.180710,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.320382,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81143,0.000,-0.307692,-0.75000,-0.178533,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0
81144,-0.250,-0.846154,0.18750,-0.238406,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0
81145,-0.375,-0.692308,-0.68750,-0.263444,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0
81146,-0.125,0.307692,-0.25000,-0.005225,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0


In [62]:
save_sparse_csr(PATH_TO_SAVE_SPARSE_FOR_CLUSTER_TO_CLASSIF, csr_matrix(matrix_for_cluster_to_classif))

## Выводы

В данном разделе были решены вопросы препроцессинга данных перед машинным обучением, а именно:

    1. Были предообработаны линейные размеры товара (были отсортированы по возрастанию, теперь для всех sku размер 'a' самый маленький, размер 'c' самый большой, это должно помочь модели находить зависимости).
    2. Был отобран датасет для обучения, чтобы каждому товару соответствовала одна упаковка. В дальнейшем, при обучении, это поможет модели более уверено предсказывать упаковку.
    3. Было проведено преобразование категориальных признаков при помощи TfIdf. Сам векторизатор был сохранён для дальнейшего использования при прогназировании.
    4. Были подготовлены датасеты для задачи классификации и кластеризации, для каждого из них был обучен и сохранён свой скейлер.